In [1]:
import pickle as pkl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from sklearn.utils import shuffle
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
resDir = r"C:\directory\results"
trainin = pkl.load(open(resDir + "\\trainin.pkl", "rb")).T
trainout = pkl.load(open(resDir + "\\trainout.pkl", "rb"))
testin = pkl.load(open(resDir + "\\testin.pkl", "rb"))
testout = pkl.load(open(resDir + "\\testout.pkl", "rb"))
yscaler = pkl.load(open(resDir + "\\yscaler.pkl", "rb"))
trainin = tf.convert_to_tensor(trainin)
trainin = tf.transpose(trainin)
trainout = tf.convert_to_tensor(trainout)
testin = tf.convert_to_tensor(testin)
testout = tf.convert_to_tensor(testout)

In [3]:
print(testin.shape)

(37, 89700)


In [4]:
def mae(y_true,y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    loss = tf.keras.losses.mean_absolute_error(y_true, y_pred)
    return K.mean(loss)

In [5]:
def mse(y_true,y_pred):
    y_pred = tf.cast(y_pred, tf.float32)
    loss = tf.keras.losses.MSE(y_true, y_pred)
    return K.mean(loss)

In [6]:
input_size = tf.transpose(trainin).shape[0]
output_size = tf.transpose(trainout).shape[0]

lr = 10e-4
epochs = 25
batchsize = 1
iterations = trainin.shape[0]//batchsize
optimizer = tf.keras.optimizers.Adam(lr, beta_1=.9, beta_2=.999, epsilon=1e-7, decay=0.)
loss_fn = mse

metric =  keras.metrics.MeanSquaredError()
val_metric = keras.metrics.MeanSquaredError()

config = {'act1': 'relu', 'act2': 'linear', 'size1': 256, 'size2': 168, 'size3':28, 'size4': 7}

inputs=keras.Input(shape = (input_size,), name = 'input')
x = Dense(int(config['size1']), input_shape = (input_size,), activation = config['act1'])(inputs)
x = Reshape(target_shape = (8, 8, 4))(x)
x = BatchNormalization()(x)
x = Conv2D(16, (3,3), activation = config['act1'], padding = "same")(x)
x = Conv2D(16, (3,3), activation = config['act1'], padding = "same")(x)
x = MaxPooling2D((2, 2), strides=2)(x)
x = Conv2D(32, (3,3), activation = config['act1'], padding = "same")(x)
x = Conv2D(32, (3,3), activation = config['act1'], padding = "same")(x)
x = MaxPooling2D((2, 2), strides=2)(x)
x = Conv2D(64, (3,3), activation = config['act1'], padding = "same")(x)
x = Conv2D(64, (3,3), activation = config['act1'], padding = "same")(x)
x = MaxPooling2D((2, 2), strides=2)(x)
x = Flatten()(x)
x = Dense(int(config['size2']), activation = config['act1'])(x)
x = BatchNormalization()(x)
x = Dense(int(config['size3']), activation = config['act1'])(x)
x = BatchNormalization()(x)
x = Dense(int(config['size4']), activation = config['act1'])(x)
x = BatchNormalization()(x)
main_output = Dense(output_size, activation = config['act2'], name = 'main_output')(x)

outputs = [main_output]

model = keras.Model(inputs = inputs, outputs = outputs)
modelname = ("BnCNN_%.1e_%f_%f" % (lr, epochs, batchsize))

In [7]:
indices = tf.range(start=0, limit=tf.shape(trainin)[0], dtype=tf.int32)

In [8]:
def train_model_on_batch():
    start = epoch * batchsize
    shuffled_indices = tf.random.shuffle(indices)
    shuffledin = tf.gather(trainin, shuffled_indices)
    shuffledout = tf.gather(trainout, shuffled_indices)
    x_batch = shuffledin[start:start + batchsize, :]
    y_batch = shuffledout[start:start + batchsize, :]
    
    with tf.GradientTape() as tape:
        if iteration % 1 == 0:
            current_loss = loss_fn(y_batch, model(x_batch))
    
    gradients = tape.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    
    metric.update_state(y_batch, model(x_batch))
    
    
    if iteration == (iterations - 1):
        val_metric.update_state(testout, model(testin))
        val_loss = loss_fn(testout, model(testin))
        return current_loss, metric.result(), val_loss, val_metric.result()
    else:
        return current_loss, 0, 0, 0

In [9]:
loss_history = []
metric_history = []
val_loss_history = []
val_metric_history = []
min_loss = 1
for epoch in range(epochs):
    for iteration in range(iterations):
        current_loss, metric_result, val_loss, val_metric_result = train_model_on_batch()
    if epoch % 1 == 0:
        loss_history.append(current_loss.numpy())
        metric_history.append(metric_result.numpy())
        val_loss_history.append(val_loss.numpy())
        val_metric_history.append(val_metric_result.numpy())
        print("\nEpoch: {}/{} - Loss: {} - MSE: {} \\n\nVal_Loss: {} - Val_MSE: {}".format(
            (epoch + 1), epochs, loss_history[-1], metric_history[-1], val_loss_history[-1], val_metric_history[-1]))
    if val_metric_result < .3:
        if val_metric_result < min_loss:
            model.save("%s\\%s.h5" % (resDir,modelname))
            min_loss = val_metric_result
    metric.reset_states()
    val_metric.reset_states()


Epoch: 1/25 - Loss: 0.07575695961713791 - MSE: 0.8103616237640381 \n
Val_Loss: 0.8798912763595581 - Val_MSE: 0.8798912763595581

Epoch: 2/25 - Loss: 0.027589742094278336 - MSE: 0.8812363147735596 \n
Val_Loss: 0.4642776846885681 - Val_MSE: 0.4642776846885681

Epoch: 3/25 - Loss: 0.001142560737207532 - MSE: 0.4799664616584778 \n
Val_Loss: 0.6766042709350586 - Val_MSE: 0.6766042709350586

Epoch: 4/25 - Loss: 0.009039019234478474 - MSE: 0.37387481331825256 \n
Val_Loss: 0.2728523313999176 - Val_MSE: 0.2728523313999176

Epoch: 5/25 - Loss: 0.002281452063471079 - MSE: 0.08532393723726273 \n
Val_Loss: 0.13897894322872162 - Val_MSE: 0.13897894322872162

Epoch: 6/25 - Loss: 0.020896898582577705 - MSE: 0.0941571295261383 \n
Val_Loss: 0.18501141667366028 - Val_MSE: 0.18501141667366028

Epoch: 7/25 - Loss: 2.1374095013015904e-05 - MSE: 0.056820277124643326 \n
Val_Loss: 0.02593529224395752 - Val_MSE: 0.02593529224395752

Epoch: 8/25 - Loss: 0.0006109442329034209 - MSE: 0.011239998042583466 \n
Val_L

In [10]:
keras.utils.plot_model(model, '%s\\customlossmodel.png' % resDir, show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [11]:
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 89700)]           0         
_________________________________________________________________
dense (Dense)                (None, 256)               22963456  
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 4)           0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 4)           16        
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 8, 16)          592       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 16)          0     

In [12]:
varnames = ["A", "Tau", "S0", "Alpha", "Phi", "Beta", "Delta S"]
pred = tf.transpose(yscaler.inverse_transform(model(testin)))
true = tf.transpose(yscaler.inverse_transform(testout))
for i in range(pred.shape[0]):
    plt.hist(pred[i], bins = 6, histtype='step', label = 'Model Pred')
    plt.hist(true[i], bins = 6, histtype = 'step', label = 'Truth Label')
    plt.title(varnames[i])
    plt.legend()
    plt.savefig(resDir + "\\%s" % varnames[i])
    plt.close()

In [13]:
print(pred.numpy()-true.numpy())

[[ 3.96337549e-03 -2.11606162e-05 -3.87439864e-05 -9.71652444e-06
   3.50221435e-03 -3.44524520e-05 -6.79589684e-06 -1.24583381e-05
  -1.49319309e-05 -3.09655803e-05 -2.76575225e-05 -2.27699416e-05
  -3.82373470e-05 -1.65225043e-04 -2.23229068e-05 -1.43358844e-05
  -1.42464774e-05  3.86771003e-03  3.82670204e-03 -1.73459190e-05
  -1.45445007e-05 -1.57961982e-05 -3.20682662e-05 -1.63326400e-05
  -2.10712093e-05 -2.69720691e-05 -4.23202651e-05 -3.23364871e-05
  -2.67932552e-05 -9.53771050e-06  3.74158660e-03 -1.62134307e-05
  -2.69720691e-05  3.54062954e-03 -1.49319309e-05 -2.05943721e-05
  -1.08490127e-05]
 [-7.35951941e-02  4.91797924e-04  1.83457136e-03 -7.00443983e-04
  -6.48343245e-02  1.43462420e-03 -8.07344913e-04 -3.72558832e-04
  -1.40100718e-04  1.18672848e-03  1.03527308e-03  4.58478928e-04
   1.68573856e-03  1.18941665e-02  3.76820564e-04 -1.14560127e-04
  -1.32739544e-04 -7.18001406e-02 -7.09965507e-02  9.76920128e-05
  -9.68873501e-05  3.23057175e-05  1.28370523e-03  2.6524

In [14]:
print(np.mean(pred.numpy()-true.numpy()))

-0.004003692930635151
